## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

Some considerations with regard to how these notebooks will be graded:

1. You can add more notebook cells or edit existing notebook cells other than "# YOUR CODE HERE" to test out or debug your code. We actually highly recommend you do so to gain a better understanding of what is happening. However, during grading, **these changes are ignored**. 
2. You must ensure that all your code for the particular task is available in the cells that say "# YOUR CODE HERE"
3. Every cell that says "# YOUR CODE HERE" is followed by a "raise NotImplementedError". You need to remove that line. During grading, if an error occurs then you will not receive points for your work in that section.
4. If your code passes the "assert" statements, then no output will result. If your code fails the "assert" statements, you will get an "AssertionError". Getting an assertion error means you will not receive points for that particular task.
5. If you edit the "assert" statements to make your code pass, they will still fail when they are graded since the "assert" statements will revert to the original. Make sure you don't edit the assert statements.
6. We may sometimes have "hidden" tests for grading. This means that passing the visible "assert" statements is not sufficient. The "assert" statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. Passing the visible tests is necessary but not sufficient to get the grade for that cell.
7. When you are asked to define a function, make sure you **don't** use any variables outside of the parameters passed to the function. You can think of the parameters being passed to the function as a hint. Make sure you're using all of those variables.
8. Finally, **make sure you run "Kernel > Restart and Run All"** and pass all the asserts before submitting. If you don't restart the kernel, there may be some code that you ran and deleted that is still being used and that was why your asserts were passing.

# Natural Language Processing

In [ ]:
# # Uncomment the below line to install
! pip install spacy
! python -m spacy download en_core_web_md

In [ ]:
import sklearn
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, HashingVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.svm import LinearSVC
import numpy as np
import en_core_web_md

In [5]:
data = fetch_20newsgroups(subset="all")

In [ ]:
print(data.DESCR)

In [7]:
text = data["data"]
target = data["target"]
print("The following are the 20 topics that an article can belong to:")
print(data["target_names"])

The following are the 20 topics that an article can belong to:
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(text, target, random_state=0)

In [9]:
print(f"The training dataset contains {len(X_train)} articles.")
print(f"The test dataset contains {len(X_test)} articles.")

The training dataset contains 14134 articles.
The test dataset contains 4712 articles.


Scikit learn implements the BoW feature representation using [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), and it also has implementations for [TF-IDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer) and [hashed vector](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html#sklearn.feature_extraction.text.HashingVectorizer) representations.

Determine the feature representations of our dataset using each of those approaches.

In [10]:
%%time
# Use English stopwords and produce a BoW representation for the data using up to trigrams
# Save the vectorizer as counter and the transformed data as X_train_bow, and X_test_bow
counter = CountVectorizer(ngram_range=(1,3), stop_words='english')
transformer = counter.fit(X_train)
X_train_bow = transformer.transform(X_train)
X_test_bow = transformer.transform(X_test)

CPU times: user 33.3 s, sys: 876 ms, total: 34.2 s
Wall time: 34.2 s


In [11]:
assert counter
assert counter.stop_words == "english"
assert counter.ngram_range == (1,3)
assert len(counter.get_feature_names()) == 3034327
assert X_train_bow.shape == (14134, 3034327)
assert X_test_bow.shape == (4712, 3034327)

Note that sklearn implements a [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and [TfidfTransformer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html). The main difference between the two is in the inputs to fitting and transforming. The [Vectorizer's fit/transform](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer.fit) take an input of text whereas the [transformer's](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer.fit) take an input of a BoW vector. Given that we already determined the BoW vectors, it would be more time efficient to use TfidfTransformer.

In [12]:
%%time
# Use the BoW representation you just created above to produce a TFIDF representation of the data
# Save the transformer to tfidfer and the transformed data as X_train_tfidf, and X_test_tfidf

tfidfer = TfidfTransformer().fit(X_train_bow)
X_train_tfidf = tfidfer.transform(X_train_bow)
X_test_tfidf = tfidfer.transform(X_test_bow)

CPU times: user 2.75 s, sys: 14.9 ms, total: 2.76 s
Wall time: 2.78 s


In [13]:
assert tfidfer
assert X_train_tfidf.shape  == (14134, 3034327)
assert X_test_tfidf.shape  == (4712, 3034327)

Now use the hashing vectorizer to do the same.

In [14]:
%%time 
# Use English stopwords and produce a Hashed vector representation for the data using up to trigrams
# Save the vectorizer as hasher and the transformed data as X_train_hash, and X_test_hash
# Make sure you set alternate_sign to False so we can use this representation with Multinomial Naive
# Bayes later in the exercise.

hasher = HashingVectorizer(ngram_range=(1,3), stop_words='english', alternate_sign=False)
transformer = hasher.fit(X_train)
X_train_hash = transformer.transform(X_train)
X_test_hash = transformer.transform(X_test)

CPU times: user 6.73 s, sys: 1.81 ms, total: 6.73 s
Wall time: 6.74 s


In [15]:
assert hasher
assert hasher.stop_words == "english"
assert hasher.ngram_range == (1,3)
assert X_train_hash.shape == (14134, 1048576)
assert X_test_hash.shape == (4712, 1048576)

Compare the time it took to run the count vectorizer vs the hasing vectorizer even though they both will iterate through all the words.

Now recall [Naive Bayes Classification](http://scikit-learn.org/stable/modules/naive_bayes.html) which we discussed early on in the supervised learning lectures. We will use Naive Bayes classifiers to predict the topic of the articles and compare our feature representations. Use a Multinomial Naive Bayes classifier to predict the topics.

In [16]:
for feat_name, train_feat, test_feat in zip(["Bag of Words", "TF-IDF", "Hashing"],[X_train_bow, X_train_tfidf, X_train_hash], [X_test_bow, X_test_tfidf, X_test_hash]):
    # Create a Multinomial Naive Bayes model saved to `mnb` and fit it to train_feat
    mnb = MultinomialNB().fit(train_feat, y_train)
    y_pred = mnb.predict(test_feat)
    print(f"Results for {feat_name}")
    print("-"*80)
    print(classification_report(y_test, y_pred))
    print("-"*80)


Results for Bag of Words
--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       205
           1       0.78      0.87      0.82       245
           2       0.92      0.76      0.83       250
           3       0.77      0.83      0.80       243
           4       0.89      0.85      0.87       255
           5       0.84      0.91      0.88       240
           6       0.90      0.75      0.82       249
           7       0.89      0.90      0.89       219
           8       0.96      0.91      0.94       246
           9       0.92      0.97      0.94       227
          10       0.96      0.98      0.97       287
          11       0.88      0.97      0.92       234
          12       0.93      0.82      0.87       247
          13       0.93      0.92      0.93       250
          14       0.90      0.96      0.93       240
          15       0.93      

In [17]:
assert isinstance(mnb, MultinomialNB)

## Learned Embeddings

We will use [spacy](https://spacy.io/) for more sophisticated NLP. Make sure you downloaded the english model in the commented code at the top of the notebook before proceeding. It may take some time to download.

Spacy allows us to parse text and automatically does the following:
- tokenization
- lemmatization
- sentence splitting
- entity recognition
- token vector representation


In [18]:
%%time
nlp = en_core_web_md.load()

CPU times: user 17.6 s, sys: 332 ms, total: 18 s
Wall time: 18 s


In [19]:
text = "This is the first sentence in this test string. The quick brown fox jumps over the lazy dog."

parsed_text = nlp(text)

In [20]:
for sent in parsed_text.sents:
    print(f"Analyzing sentence: {sent}")
    print(f"Lemmatization: {sent.lemma_}")
    for token in sent:
        print(f"Analyzing token: {token}")
        if token.is_sent_start:
            print("This token is the first one in the sentence")
        if token.is_stop:
            print("Stop word")
        else:
            print("Not stop word")
        print(f"Entity type: {token.ent_type_}")
        print(f"Part of speech: {token.pos_}")
        print(f"Lemma: {token.lemma_}")
        print("-"*10)
    print("-"*50)

Analyzing sentence: This is the first sentence in this test string.
Lemmatization: this be the first sentence in this test string .
Analyzing token: This
This token is the first one in the sentence
Stop word
Entity type: 
Part of speech: DET
Lemma: this
----------
Analyzing token: is
Stop word
Entity type: 
Part of speech: AUX
Lemma: be
----------
Analyzing token: the
Stop word
Entity type: 
Part of speech: DET
Lemma: the
----------
Analyzing token: first
Stop word
Entity type: ORDINAL
Part of speech: ADJ
Lemma: first
----------
Analyzing token: sentence
Not stop word
Entity type: 
Part of speech: NOUN
Lemma: sentence
----------
Analyzing token: in
Stop word
Entity type: 
Part of speech: ADP
Lemma: in
----------
Analyzing token: this
Stop word
Entity type: 
Part of speech: DET
Lemma: this
----------
Analyzing token: test
Not stop word
Entity type: 
Part of speech: NOUN
Lemma: test
----------
Analyzing token: string
Not stop word
Entity type: 
Part of speech: NOUN
Lemma: string
--------

In [21]:
### Come up with a couple sentences to test out and set the text to my_text
### You can go to your favorite website or news source and copy a paragraph from there

my_text = """We may sometimes have hidden tests for grading. This means that passing the visible assert statements is not sufficient. 
The assert statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. 
Passing the visible tests is necessary but not sufficient to get the grade for that cell."""

In [22]:
assert len(my_text) > 10
assert my_text.count(".") > 2

In [23]:
parsed = nlp(my_text)
for sent in parsed.sents:
    print(f"Analyzing sentence: {sent}")
    print(f"Lemmatization: {sent.lemma_}")
    for token in sent:
        print(f"Analyzing token: {token}")
        if token.is_sent_start:
            print("This token is the first one in the sentence")
        if token.is_stop:
            print("Stop word")
        else:
            print("Not stop word")
        print(f"Entity type: {token.ent_type_}")
        print(f"Part of speech: {token.pos_}")
        print(f"Lemma: {token.lemma_}")
        print("-"*10)
    print("-"*50)

Analyzing sentence: We may sometimes have hidden tests for grading.
Lemmatization: -PRON- may sometimes have hide test for grading .
Analyzing token: We
This token is the first one in the sentence
Stop word
Entity type: 
Part of speech: PRON
Lemma: -PRON-
----------
Analyzing token: may
Stop word
Entity type: 
Part of speech: VERB
Lemma: may
----------
Analyzing token: sometimes
Stop word
Entity type: 
Part of speech: ADV
Lemma: sometimes
----------
Analyzing token: have
Stop word
Entity type: 
Part of speech: AUX
Lemma: have
----------
Analyzing token: hidden
Not stop word
Entity type: 
Part of speech: VERB
Lemma: hide
----------
Analyzing token: tests
Not stop word
Entity type: 
Part of speech: NOUN
Lemma: test
----------
Analyzing token: for
Stop word
Entity type: 
Part of speech: ADP
Lemma: for
----------
Analyzing token: grading
Not stop word
Entity type: 
Part of speech: NOUN
Lemma: grading
----------
Analyzing token: .
Not stop word
Entity type: 
Part of speech: PUNCT
Lemma: .
-

If we use the larger spacy models, we get the GloVe representation for some words based on a pre-trained model. The GloVe vectors should be in 300 dimensions.

In [24]:
token.vector.shape

(300,)

Given that the parsing of text takes some time, we will only consider the first 1000 articles in our data.

In [25]:
new_X_train, new_X_test, new_y_train, new_y_test = train_test_split(X_train[:1000], y_train[:1000], random_state=0)

In [26]:
%%time
# Using nlp from above, parse every instance of new_X_train
# save the document vectors to a np.array called X_train_glove

X_train_glove = np.empty((len(new_X_train), token.vector.shape[0]))
X_test_glove = np.empty((len(new_X_test), token.vector.shape[0]))

for i in range(len(new_X_train)):
  temp_text = new_X_train[i]
  parsed = nlp(temp_text)
  X_train_glove[i,:] = parsed.vector
for i in range(len(new_X_test)):
  temp_text = new_X_test[i]
  parsed = nlp(temp_text)
  X_test_glove[i,:] = parsed.vector

CPU times: user 1min 7s, sys: 2.76 s, total: 1min 10s
Wall time: 1min 10s


In [27]:
assert X_train_glove.shape == (len(new_X_train), 300)
assert X_test_glove.shape == (len(new_X_test), 300)

In [28]:
svm = LinearSVC().fit(X_train_glove, new_y_train)
y_pred = svm.predict(X_test_glove)
print(classification_report(new_y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.50      0.50      0.50        10
           2       0.60      0.55      0.57        11
           3       0.60      0.50      0.55        12
           4       0.50      0.50      0.50        12
           5       0.58      0.64      0.61        11
           6       0.64      0.90      0.75        10
           7       0.77      0.94      0.85        18
           8       0.88      0.82      0.85        17
           9       0.76      0.76      0.76        17
          10       0.64      0.64      0.64        14
          11       0.85      0.79      0.81        14
          12       0.69      0.47      0.56        19
          13       0.80      1.00      0.89        16
          14       0.75      0.60      0.67        10
          15       0.56      0.77      0.65        13
          16       0.92      0.79      0.85        14
          17       0.73    

We will not cover LDA in this exercise but if you are interested in topic modeling, you should check out [Gensim](https://radimrehurek.com/gensim/) and its [LDA implementation](https://radimrehurek.com/gensim/models/ldamodel.html).

## Feedback

In [29]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    return "All good!"

In [30]:
print(feedback())

All good!
